# Shear Likelihood 1.2

### Preparing necessary libraries

In [8]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from numpy import random
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units

### Creating mock data

In [9]:
np.random.seed(11)

cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

cluster_m     = 1.e15
logm = np.log(cluster_m)/np.log(10)
cluster_z     = 0.4
concentration = 4
ngals         = 10000
Delta         = 200
cluster_ra    = 20.0
cluster_dec   = 90.0

ideal_data   = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 0.8, zsrc_min = cluster_z + 0.1, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
ideal_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo,'chang13', zsrc_min = cluster_z + 0.1, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
noisy_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1, shapenoise=0.05, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)

gc_ideal   = clmm.GalaxyCluster("CL_ideal", cluster_ra, cluster_dec, cluster_z, ideal_data)
gc_ideal_z = clmm.GalaxyCluster("CL_ideal_z", cluster_ra, cluster_dec, cluster_z, ideal_data_z)
gc_noisy_z = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, noisy_data_z)

# For some reason galaxy catalogs access information very deifferently from galaxy clusters

### Calculating observables

In [10]:
theta_ideal, gt_ideal, gx_ideal       = gc_ideal.compute_tangential_and_cross_components(geometry="flat")
theta_ideal_z, gt_ideal_z, gx_ideal_z = gc_ideal_z.compute_tangential_and_cross_components(geometry="flat")
theta_noisy_z, gt_noisy_z, gx_noisy_z = gc_noisy_z.compute_tangential_and_cross_components(geometry="flat")

# Convert from theta to radius
radius_ideal, radius_ideal_z, radius_noisy_z = [convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=cluster_z, cosmo=cosmo) for gc in [gc_ideal, gc_ideal_z, gc_noisy_z]]

### Binning the data

In [11]:
bin_edges = da.make_bins(0.7, 4, 15, method='evenlog10width')

profile_ideal   = gc_ideal.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_ideal_z = gc_ideal_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_noisy_z = gc_noisy_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

profiles = [profile_ideal, profile_ideal_z, profile_noisy_z]

/home/caiooliveira/.local/lib/python3.10/site-packages/clmm-1.1.10-py3.10.egg/clmm/utils.py:176: RuntimeWarning: invalid value encountered in sqrt


### Calculating Likelihood

The measured shear $g^i_t$ is given by 

$$ g^i_t = g^{I,i}_t + g^{L,i}_t $$

where $g^{I,i}_t$ is the intrisic shear and $g^{L,i}_t$ is the lensing shear. Assuming that the intrinsic shear is drawn from a normal distribution with average 0 and variance $\sigma_{g^I_t}^2$, the probability of measuring $g^i_t$, given a certain $r$, $z$ and other cosmological parameters $\vec{\theta_c}$ is given by

$$ P(g^i_t | r, z, \vec{\theta_c}) = \frac{1}{\sqrt{2\pi}\sigma_{g^I_t}} \exp\left[\frac{-(g^i_t - g^{L,i}_t(r,z,\vec{\theta_c}))^2}{2\sigma_{g^I_t}^2}\right] $$
Then, to find the likelihood of the set of all $\{g^i_t\}_i^{N}$ measured, we perform the following integral for all $g^i_t$:

$$ P(g^i_t | \vec{\theta_c}) = \int_0^{r_{max}} \int_{z_{cluster}}^{\infty} \mathcal{d}r \mathcal{d}z P(g^i_t | r, z, \vec{\theta_c}) P(r) P(z) $$

where $P(r)$ and $P(z)$ are the distributions for $r$ and $z$.

This is a computationally expensive process though. To avoid it, we bin the data. We then have to perform the same integrals, but for each bin and each shear $g^i_t$ in the bin. At this moment we'll choose to not calculate them, and instead use an estimator for the probability

$$ \tilde{P}(g^i_t | \vec{\theta_c}) = \frac{1}{N_i} \sum_j^{N_i} P(g^i_t | r_j, z_j, \vec{\theta_c}) $$

where $N_i$ is the number of galaxies in the bin. The bin likelihood is then

$$ \mathcal{L}_i = \prod_j^{N_i} \tilde{P}(g^j_t | \vec{\theta_c}) $$

and the total likelihood is 

$$ \mathcal{L} = \prod_i^M \mathcal{L}_i = \prod_i^M \prod_j^{N_i} \tilde{P}(g^j_t | \vec{\theta_c}) =  \prod_i^M \prod_j^{N_i} \left[ \frac{1}{N_i} \sum_k^{N_i} P(g^j_t | r_k, z_k, \vec{\theta_c}) \right]  $$

In [12]:
# Probability of measuring gt_obs given a certain r, z, cosmo, etc.
def shear_probability(r, gal_z, cluster_z, gt_obs, sigma, cosmo, m=1.e15, concentration=4, delta=200, model='nfw'):

    gt_th = clmm.compute_reduced_tangential_shear(r, m, concentration, cluster_z, gal_z, cosmo, delta_mdef=200, halo_profile_model=model)

    return np.exp(-np.power(np.subtract(gt_obs, gt_th), 2)/2/sigma**2)/np.sqrt(2*np.pi)/sigma

In [13]:
def shear_log_likelihood(radius, gt, gc_data, cosmo, sigma=1, m=1.e15, concentration=4, delta=200, model='nfw'):

    n_bins = len(gc_data.profile)
    cluster_z = gc_data.z
    likelihood = 0

    for i in range(n_bins):

        gal_list = gc_data.profile['gal_id'][i]
        z_list = gc_data.galcat['z'][gal_list]
        r_list = [radius[gal_id] for gal_id in gal_list]
        gt_prob_bin = 0

        for gal in gal_list:

            gt_obs_gal = gt[gal]
            gt_prob = np.mean(shear_probability(r_list, z_list, cluster_z, gt_obs_gal, sigma, cosmo, m, concentration, delta, model))
            gt_prob_bin += np.log(gt_prob)

        likelihood += gt_prob_bin

    return -2*likelihood

In [14]:
shear_log_likelihood(radius_ideal_z, gt_ideal_z, gc_ideal_z, cosmo)

14002.640036901772